In [ ]:
import json

with open('oa_driving_test_pcap_20240827.json', 'r') as f:
    pcap = json.load(f)


#list variables for the core data and partII data
coreData_list =[]
PartII_list = []

#merged data of core data aand part II data
BSM_data_list = []

#access data inside json file and retrieve the BSM data
for data in pcap:
  #access the Basic Safety Message when there is a wsmp value in the dictionary
  if 'wsmp' in data['_source']['layers']:
    Wave_short_message = data['_source']['layers']["wsmp"]["Wave Short Message"]
    content_tree = Wave_short_message["ieee1609dot2.Ieee1609Dot2Data_element"]["ieee1609dot2.content_tree"]
    signedData = content_tree["ieee1609dot2.signedData_element"]
    tbsData = signedData["ieee1609dot2.tbsData_element"]
    payload = tbsData["ieee1609dot2.payload_element"]
    data_element = payload["ieee1609dot2.data_element"]
    content_tree_1 = data_element["ieee1609dot2.content_tree"]
    j2735 = content_tree_1["j2735"]
    MessageFrame = j2735["j2735.MessageFrame_element"]
    value_element = MessageFrame["j2735.value_element"]
    basicsafetymessage_element = value_element["j2735.BasicSafetyMessage_element"]

    #open core data sequence
    coreData_element_value = basicsafetymessage_element["j2735.coreData_element"]
    coreData_element = {'Core Data': coreData_element_value}
    coreData_list.append(coreData_element)

    #open partII sequence
    partII = basicsafetymessage_element["j2735.partII_tree"]["Item 0"]["j2735.PartIIcontent_element"]['j2735.partII_Value_element']
    partII_safety_extension_value = partII["j2735.VehicleSafetyExtensions_element"]
    partII_safety_extension = {'vehicleSafetyExt': partII_safety_extension_value}
    partII_value = {'Part II':partII_safety_extension}
    PartII_list.append(partII_value)

    #merge both dictionaries into one
    BSM_merge = {**coreData_element, **partII_value}
    BSM_data_list.append(BSM_merge)

#write a new json file to store the merged dictionary
with open('BSM_list.json', 'w') as f:
   json.dump(BSM_data_list, f, indent=2)

#first iteration of the level in the dictionary test
BSM_units = BSM_data_list[0]

#reccursive formula to display all the key value of the merged file
def get_all_key_values(dictionary, level=1):
    for key, value in dictionary.items():
        if isinstance(value, dict):
            print('   ' * (level - 1) + f'[{level}] {key}') #if the dictionary still contains dictionary print key only
            get_all_key_values(value, level + 1) #reccursion
        else:
            print('   ' * (level - 1) + f'[{level}] {key} : {value}') #if the dictionary contains no more dictionary, print key and value

#for all iteration in the json file
for data in BSM_data_list:
  get_all_key_values(data)
